# Confidence Intervals
<hr>

A confidence interval is a range is which we are confident to a certain degree that a specific population parameter will fall. 

For example, according to [glassdoor](https://www.glassdoor.com/Salaries/data-scientist-salary-SRCH_KO0,14.htm), as of April 7th 2020, the mean salary for data scientists is \\$113,309. Now obviously glassdoor hasn't polled every working data scientist that exists to get to this number, meaning \\$113,309 is an estimate of the true mean.

So if we were to construct a 95% CI for this average and (just making up the numbers) got \\$100,000 - \\$115,000. This means that if we were to poll many times the average salary of __random__ samples of data scientists, than in 95% of those cases we would get an average salary that falls in that range.

Please note most of the data used in this notebook for the examples originates from [this udemy course](https://www.udemy.com/course/the-data-science-course-complete-data-science-bootcamp/). I highly recommend it!

In [58]:
import pandas as pd
import numpy as np
# we'll use both a normal distribution and a students T distribution
from scipy.stats import norm, t

## General notes

__Note:__ In this notebook every $\sigma$ represents a population standard deviation while $s$ represents a sample standard deviation. Also $\mu$ generally denotes the mean of some population, while $\bar{x}$ denotes a sample mean.

CI's can either be two-sided or one-sided. A two sided CI means a population parameter falls in between two values i.e.

$$
\mu \in \bar{x} \pm \text{ME}
$$

while a one sided CI could either mean 
$$
\mu \le \bar{x} + \text{ME} \\
\mu \ge \bar{x} + \text{ME}
$$

where $\text{ME}$ = Margin of Error.

Another thing to be aware of in calculating CI's is when to use a Z (normal) distribution or a T distribution. In general when the population variance is known or the data you have is sufficiently big enough (usually 30+ samples is a good rule of thumb) you use a Z distribution. Otherwise use a T distribution which is meant for smaller amounts of data.

Lastly when using a T distribution you need to specify how many [Degrees of Freedom](https://en.wikipedia.org/wiki/Degrees_of_freedom_(statistics)) it has. 

Which in general is just $\text{the number of data points in the set} - 1$.

Now let's get into it.

## CI - Population variance known

$$
\large \bar{x} \pm z_{\alpha/2} * \frac{\sigma}{\sqrt{n}}
$$
Let's break this down:

    α is known as the confidence level and is just 1 - confidence. For a confidence of 90%, α = 10% or 0.1, for a  confidence of 95%, α = 0.05, and for a confidence of 99%, α = 0.01
    
    n is the number of samples in our data.

Let's calculate a 95% CI using a dataset with the salaries of 70 data scientists. __We'll assume for the sake of this example that the population salary standard deviation is $15,000.__

In [59]:
data = pd.read_csv('data/salaries_extended.csv')
print(f'There are {len(data)} samples in the dataset.')
data

There are 70 samples in the dataset.


,Salaries
0,120643
1,131248
2,108833
3,127776
4,114564
...,...
65,112276
66,85927
67,102848
68,121200


In [60]:
# In jupyter you can easily type 
# any greek symbol like α by typing
# \alpha and hitting the tab key.
α = 0.05
n = len(data)

mean_salary = data.Salaries.mean()
σ = 15_000

print(f'The sample mean salary is ${mean_salary:,.2f}')
print(f'The population standard deviation is ${σ:,}')

# note we're using a normal distribution
# ppf is just the inverse of the cdf
z_score = norm.ppf(1-α/2)
print(f'\nThe z-score for a {int((1-α)*100)}% CI is {z_score:.2f}')

print('\n', '='*45, '\n', sep='')

lower_bound = mean_salary - z_score * σ / np.sqrt(n)
upper_bound = mean_salary + z_score * σ / np.sqrt(n)

print(f'The {int((1-α)*100)}% CI ranges from ${lower_bound:,.2f} to ${upper_bound:,.2f}')

The sample mean salary is $113,309.00
The population standard deviation is $15,000

The z-score for a 95% CI is 1.96


The 95% CI ranges from $109,795.09 to $116,822.91


We see for a 95% CI we get \\$109,795.09 to \\$116,822.91 this means that we are __confident__ that in 95% of the time the true mean will fall in this range. It doesn't mean that exactly 95% of the time that will happen just that we are confident that it will.

Also note even though the formula says $z_{\alpha/2}$ in practice we use $z_{1-\alpha/2}$ because
$$\large z_{1-\alpha/2} = -z_{\alpha/2} $$

This is true because a normal distribution is symmetric around its mean. Using either one yields the same result but using $z_{\alpha/2}$, you'd have to swap the `lower_bound` and `upper_bound` variables.

<br>

<hr>

## CI - Population variance unknown

Since we don't know the population variance and the dataset is pretty small, we use a Student's t distribution.
Also because we don't know the population variance we need to estimate the standard deviation using the sample standard deviation $s$.

$$
\large \mu \in \bar{x} \pm t_{n-1,\alpha/2} * \frac{s}{\sqrt{n}}
$$

$t_{n-1,\alpha/2}$ indicates a t-score from a t distribution with $n - 1$ degrees of freedom and a cdf value of $\alpha/2$

Note: $s\;/\sqrt{n}$ is also known as the __standard error__.
<hr>
Recall the difference between population standard deviation $\sigma$ and the sample standard deviation $s$:

$$
\sigma = \sqrt \frac{\Sigma_{i=1}^n (x_i - \mu)^2}{n} \;\;\;\;\;,\;\;\;\;\;   
s = \sqrt \frac{\Sigma_{i=1}^n (x_i - \bar x)^2}{n-1}
$$
where $x_i$ is each value in the population, $\mu$ is the mean, $\bar x$ is the sample mean, and $n$ is the number of values.

In [61]:
data = pd.read_csv('data/salaries_popvar_unknown.csv')
data

,Salaries
0,78000
1,90000
2,75000
3,117000
4,105000
5,96000
6,89500
7,102300
8,80000


In [62]:
# This time let's calculate a 99% CI
α = 0.01
n = len(data)

mean_salary = data.Salaries.mean()

# we can calculate the sample std by hand
sample_var = np.square(data.Salaries - mean_salary).sum() / (n - 1)
s = np.sqrt(sample_var)
print(s)

# or just have pandas do it for us
print(data.Salaries.std())

13931.887883556916
13931.887883556918


In [63]:
print(f'The sample mean salary is ${mean_salary:,.2f}')
print(f'The sample standard deviation of the salaries is ${s:,.2f}')

# note we're using a t distribution
t_score = t.ppf(1-α/2, df=n-1)
print(f'\nThe t-score for a {int((1-α)*100)}% CI is {t_score:.2f}')

print('\n', '='*45, '\n', sep='')

lower_bound = mean_salary - t_score * s / np.sqrt(n)
upper_bound = mean_salary + t_score * s / np.sqrt(n)

print(f'The {int((1-α)*100)}% CI ranges from ${lower_bound:,.2f} to ${upper_bound:,.2f}')

The sample mean salary is $92,533.33
The sample standard deviation of the salaries is $13,931.89

The t-score for a 99% CI is 3.36


The 99% CI ranges from $76,951.04 to $108,115.63


## CI - Mean of differences for dependent samples

We'll start with an example: Let's say a medical group is testing the effectiveness of a new drug that's supposed to increase the levels of Magnesium in a person. They measured each patients levels before and after taking the drug. 

In this case the samples are dependent because each sample came from the same person.

$$
\large \mu_{\text{diff}} \in \bar{d} \pm t_{n-1, \alpha/2} * \frac{s_d}{\sqrt{n}}
$$

In [64]:
data = pd.read_csv('data/mean_diff_dependent.csv', index_col='Patient')
data

,Before,After
Patient,,
1,2.0,1.7
2,1.4,1.7
3,1.3,1.8
4,1.1,1.3
5,1.8,1.7
6,1.6,1.5
7,1.5,1.6
8,0.7,1.7
9,0.9,1.7


In [65]:
# note the order: After - Before 
# positive values indicate an increase
diff = data.After - data.Before
data['Difference'] = diff
data

,Before,After,Difference
Patient,,,
1,2.0,1.7,-0.3
2,1.4,1.7,0.3
3,1.3,1.8,0.5
4,1.1,1.3,0.2
5,1.8,1.7,-0.1
6,1.6,1.5,-0.1
7,1.5,1.6,0.1
8,0.7,1.7,1.0
9,0.9,1.7,0.8


In [66]:
n = len(data)

mean_diff = diff.mean()
print(f'The mean of the differences is {mean_diff:.3f}')

std_diff = diff.std()
print(f'The standard deviation of the differences is {std_diff:.3f}')

The mean of the differences is 0.330
The standard deviation of the differences is 0.455


We can calculate the standard error manually by doing 
```python
std_diff / np.sqrt(n)
```
or we can just let pandas do it:
```python
data.Difference.sem()
```
either way the choice is yours I'm going to keep using the first example to illustrate the formula as clearly as possible.

In [67]:
assert std_diff / np.sqrt(n) == data.Difference.sem()

In [68]:
# we'll calculate a 90% CI
α = 0.1

t_score = t.ppf(1-α/2, n-1)
print(f'The t-score for a {int((1-α)*100)}% CI is {t_score:.2f}')

The t-score for a 90% CI is 1.83


In [69]:
lower_bound = mean_diff - t_score * std_diff / np.sqrt(n)
upper_bound = mean_diff + t_score * std_diff / np.sqrt(n)

print(f'The {int((1-α)*100)}% CI ranges from {lower_bound:,.3f} to {upper_bound:,.3f}')

print('''
Since all the values in the interval are positive
we can say with 90% confidence that the drug works.

If you try with a 95% CI we can still make the assumption
that the drug actually does increase magnesium levels.

But we can't say anything with 99% confidence since part 
of the interval is negative.''')

The 90% CI ranges from 0.066 to 0.594

Since all the values in the interval are positive
we can say with 90% confidence that the drug works.

If you try with a 95% CI we can still make the assumption
that the drug actually does increase magnesium levels.

But we can't say anything with 99% confidence since part 
of the interval is negative.


## CI - difference of means for independent samples with known population variances

In this case were comparing the means of two independent groups (let's say the average test scores between management students and engineering students) and we want to see if there's a significant difference in the means ie if one group does better on average than the other.

Assume the dataset looks like:

<table>
    <thead>
        <td></td>
        <td>Engineering</td>
        <td>Management</td>
    </thead>
    <tbody>
        <tr>
            <td>Size</td>
            <td>100</td>
            <td>70</td>
        </tr>
        <tr>
            <td>Sample Mean</td>
            <td>58</td>
            <td>65</td>
        </tr>
        <tr>
            <td>Population Std</td>
            <td>10</td>
            <td>5</td>
        </tr>
    </tbody>
</table>

The variance for the differnece in means is calculated by:
$$
\large \sigma_{\text{diff}}^2 = \sqrt{\frac{\sigma_x^2}{n_x} + \frac{\sigma_y^2}{n_y}}
$$

The CI is calculated by:
$$
\large \mu_{\text{diff}} \in (\bar{x} - \bar{y}) \pm z_{\alpha/2} * \sqrt{\sigma_{\text{diff}}^2}
$$

In [70]:
num_E = 100
num_M = 70

mean_E = 58
mean_M = 65

σ_E = 10
σ_M = 5

In [71]:
# 95% CI
α = 0.05

mean_diff = mean_E - mean_M
print('Difference between Engineering and Management:', mean_diff)

z_score = norm.ppf(1 - α/2)
print(f'z-score for a {int((1-α)*100)}% CI: {z_score:.2f}')

Difference between Engineering and Management: -7
z-score for a 95% CI: 1.96


In [72]:
lower_bound = mean_diff - z_score * np.sqrt(σ_E**2/num_E + σ_M**2/num_M)
upper_bound = mean_diff + z_score * np.sqrt(σ_E**2/num_E + σ_M**2/num_M)

print(f'The {int((1-α)*100)}% CI ranges from {lower_bound:,.3f} to {upper_bound:,.3f}')

print(f'''
This tells us that we are confident that {int((1-α)*100)}%
of the time Engineering students score {-upper_bound:,.3f}
to {-lower_bound:,.3f} points less on exams than management
students.''')

The 95% CI ranges from -9.283 to -4.717

This tells us that we are confident that 95%
of the time Engineering students score 4.717
to 9.283 points less on exams than management
students.


## CI - difference of two means for independent samples with unknown variances but assumed to be equal

If we're comparing the means of two populations where we don't know the individual pop. variances but we can make the assumption that they're equal then we pool the variances. 

__pooled sample variance:__
$$
\large s_p^2 = \frac{(n_x - 1)s_x^2 + (n_y - 1)s_y^2}{n_x + n_y - 2}
$$

If there aren't a lot of samples we use a t-statistic with <br> $n_x + n_y - 2$ degrees of freedom and the interval is computed by:

$$
\large \mu_{\text{diff}} \in (\bar{x} - \bar{y}) \pm t_{n_x + n_y - 2, \alpha/2} * \sqrt{\frac{s_p^2}{n_x} + \frac{s_p^2}{n_y}}
$$

In the example we're comparing the price of apples in New York vs LA.

In [73]:
df = pd.read_csv('data/apples.csv')
df

,NY apples,LA apples
0,3.80,3.02
1,3.76,3.22
2,3.87,3.24
3,3.99,3.02
4,4.02,3.06
5,4.25,3.15
6,4.13,3.81
7,3.98,3.44
8,3.99,NaN
9,3.62,NaN


In [80]:
# 95% CI
α = 0.05

num_NY = 10
num_LA = 8

mean_NY = df['NY apples'].mean()
mean_LA = df['LA apples'].mean()
print(f'Mean price of apples in NY: ${mean_NY:.2f}')
print(f'Mean price of apples in LA: ${mean_LA:.2f}\n')

mean_diff = mean_NY - mean_LA
print(f'Difference in mean price of apples in NY vs LA: ${mean_diff:.2f}\n')

var_NY = df['NY apples'].var()
var_LA = df['LA apples'].var()
print(f'Sample variance of price of apples in NY: $^2 {var_NY:.2f}')
print(f'Sample variance of price of apples in LA: $^2 {var_LA:.2f}\n')

pooled_var = ((num_NY - 1)*var_NY + (num_LA - 1)*var_LA) / (num_NY + num_LA - 2)
print(f'Pooled variance: $^2 {pooled_var:.2f}')

Mean price of apples in NY: $3.94
Mean price of apples in LA: $3.25

Difference in mean price of apples in NY vs LA: $0.70

Sample variance of price of apples in NY: $^2 0.03
Sample variance of price of apples in LA: $^2 0.07

Pooled variance: $^2 0.05


In [81]:
t_score = t.ppf(1-α/2, num_NY+num_LA-2)
t_score

2.1199052992210112

In [82]:
lower_bound = mean_diff - t_score * np.sqrt(pooled_var/num_NY + pooled_var/num_LA)
upper_bound = mean_diff + t_score * np.sqrt(pooled_var/num_NY + pooled_var/num_LA)

print(f'The {int((1-α)*100)}% CI ranges from ${lower_bound:.2f} to ${upper_bound:.2f}')
print(f'''
We have {int((1-α)*100)}% confidence that apples in NY
are ${lower_bound:.2f} to ${upper_bound:.2f} more expensive than
apples in LA.''')

The 95% CI ranges from $0.47 to $0.92

We have 95% confidence that apples in NY
are $0.47 to $0.92 more expensive than
apples in LA.
